# Miniprot mapping
Setting up the required Python packages

In [ ]:
import os
import pandas as pd
import subprocess
import time
from pathlib import Path

from spider_silkome_module import (
    RAW_DATA_DIR,
    INTERIM_DATA_DIR,
    EXTERNAL_DATA_DIR,
    PROCESSED_DATA_DIR
)
from spider_silkome_module.utils import run_cmd

## Configure

Definition of the path for relevant data

In [19]:
task_name = "miniprot_mapping_20260127"

spidroin_fasta_file = EXTERNAL_DATA_DIR / "spider-silkome-database.v1.prot.fixed.renamed.fasta"
spider_genome_path = RAW_DATA_DIR / "spider_genome"

# dedup
dedup_output_dir = INTERIM_DATA_DIR / task_name / "cdhit"
spidroin_fasta_file_rep = dedup_output_dir / "cdhit_rep_seq.fa"
shortest_seq_file = dedup_output_dir / "cdhit_shortest_seq.fa"

# miniprot
miniprot_output_path = PROCESSED_DATA_DIR / task_name / "miniprot_output"



Perform redundancy reduction on the N/C terminal sequences of Spidroin proteins using cd-hit.

<zh>使用 cd-hit 对 Spidroin 蛋白的 N/C 端序列进行去冗余。</zh>

In [17]:

cdhit_cmd = f"pixi run cd-hit -i {spidroin_fasta_file} -o {spidroin_fasta_file_rep} -c 0.9 -T 0 -M 0 -d 0"
shortest_seq_cmd = f"pixi run python -m spider_silkome_module.extract_shortest_seq --clstr-path {spidroin_fasta_file_rep}.clstr --fasta-path {spidroin_fasta_file} --output-path {shortest_seq_file}"
run_cmd(cdhit_cmd, [spidroin_fasta_file_rep])
run_cmd(shortest_seq_cmd, [shortest_seq_file],force=True)

⏭️ /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_rep_seq.fa exists, skip


2026-01-27 10:51:07.499 | INFO     | spider_silkome_module.config:<module>:11 - PROJ_ROOT path is: /home/gyk/project/spider_silkome


2026-01-27 10:51:07.725 | INFO     | __main__:main:61 - Parsing cluster file: /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_rep_seq.fa.clstr
2026-01-27 10:51:07.752 | INFO     | __main__:main:63 - Found 6985 clusters
2026-01-27 10:51:07.752 | DEBUG    | __main__:get_shortest_seq_ids:48 - Cluster 0: shortest = 5826_MaSp2_CTD (1854aa)
2026-01-27 10:51:07.752 | DEBUG    | __main__:get_shortest_seq_ids:48 - Cluster 1: shortest = 2571_Flag_CTD (448aa)
2026-01-27 10:51:07.752 | DEBUG    | __main__:get_shortest_seq_ids:48 - Cluster 2: shortest = 1737_MiSp_CTD (1674aa)
2026-01-27 10:51:07.752 | DEBUG    | __main__:get_shortest_seq_ids:48 - Cluster 3: shortest = 4955_AgSp1_CTD (200aa)
2026-01-27 10:51:07.752 | DEBUG    | __main__:get_shortest_seq_ids:48 - Cluster 4: shortest = 4681_AgSp1_CTD (200aa)
2026-01-27 10:51:07.752 | DEBUG    | __main__:get_shortest_seq_ids:48 - Cluster 5: shortest = 5222_AgSp1_CTD (926aa)
2026-01-27 10:51:07.752 | DEBUG    | __main

Extracting sequences: 100%|██████████| 6985/6985 [00:00<00:00, 938922.97it/s]


Use miniprot to align the C-terminal and N-terminal sequences of the spidroin gene sequences to the genome of the new species.

In [21]:
# index_cmd = f"miniprot -t70 -d {gnome_mpi_path}/{spider}.mpi {spider_genome_path}/{spider_genome}"
miniprot_cmd = f"pixi run python -m spider_silkome_module.run_miniprot --input-path {spider_genome_path} --protein-fasta {shortest_seq_file} --output-path {miniprot_output_path} --threads 70 --outc 0.8 --force"

# run_cmd(index_cmd, [gnome_mpi_path])
run_cmd(miniprot_cmd, [miniprot_output_path])

2026-01-27 12:35:50.046 | INFO     | spider_silkome_module.config:<module>:11 - PROJ_ROOT path is: /home/gyk/project/spider_silkome
Processing genomes:   0%|          | 0/10 [00:00<?, ?it/s]

2026-01-27 12:35:50.077 | INFO     | __main__:main:76 - Input path: /home/gyk/project/spider_silkome/data/raw/spider_genome
2026-01-27 12:35:50.078 | INFO     | __main__:main:77 - Protein FASTA: /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_shortest_seq.fa
2026-01-27 12:35:50.079 | INFO     | __main__:main:84 - Found 10 genome(s) to process
2026-01-27 12:35:50.080 | INFO     | __main__:main:89 - Processing Songthela_sp: /home/gyk/project/spider_silkome/data/raw/spider_genome/049.Songthela_sp/Songthela_sp.mpi


[M::mp_idx_restore@18.123*0.56] loaded the index
[M::mp_idx_print_stat] 2398163 distinct k-mers; mean occ of infrequent k-mers: 507.90; 2150 frequent k-mers accounting for 105152995 occurrences
[M::worker_pipeline::27.231*22.50] mapped 4066 sequences
[M::worker_pipeline::29.724*25.90] mapped 2919 sequences
[M::main] Version: 0.18-r281
[M::main] CMD: miniprot -S --gff-delim . --outc 0.8 -t 70 -I --gff-only /home/gyk/project/spider_silkome/data/raw/spider_genome/049.Songthela_sp/Songthela_sp.mpi /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_shortest_seq.fa
[M::main] Real time: 30.217 sec; CPU: 770.238 sec; Peak RSS: 14.456 GB
Processing genomes:  10%|█         | 1/10 [00:30<04:32, 30.28s/it]

2026-01-27 12:36:20.357 | INFO     | __main__:main:99 - Output: /home/gyk/project/spider_silkome/data/processed/miniprot_mapping_20260127/miniprot_output/Songthela_sp/Songthela_sp.gff
2026-01-27 12:36:20.358 | INFO     | __main__:main:89 - Processing Araneus_ventricosus: /home/gyk/project/spider_silkome/data/raw/spider_genome/064.Araneus_ventricosus/Araneus_ventricosus.mpi


[M::mp_idx_restore@10.371*0.60] loaded the index
[M::mp_idx_print_stat] 2371156 distinct k-mers; mean occ of infrequent k-mers: 368.91; 368 frequent k-mers accounting for 10236796 occurrences
[M::worker_pipeline::16.531*24.23] mapped 4066 sequences
[M::worker_pipeline::18.639*28.92] mapped 2919 sequences
[M::main] Version: 0.18-r281
[M::main] CMD: miniprot -S --gff-delim . --outc 0.8 -t 70 -I --gff-only /home/gyk/project/spider_silkome/data/raw/spider_genome/064.Araneus_ventricosus/Araneus_ventricosus.mpi /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_shortest_seq.fa
[M::main] Real time: 18.983 sec; CPU: 539.287 sec; Peak RSS: 9.306 GB
Processing genomes:  20%|██        | 2/10 [00:49<03:09, 23.70s/it]

...

2026-01-27 12:41:13.746 | INFO     | __main__:main:99 - Output: /home/gyk/project/spider_silkome/data/processed/miniprot_mapping_20260127/miniprot_output/Pandercetes_sp/Pandercetes_sp.gff
2026-01-27 12:41:13.747 | INFO     | __main__:main:89 - Processing Hippasa_lycosina: /home/gyk/project/spider_silkome/data/raw/spider_genome/017.Hippasa_lycosina/Hippasa_lycosina.mpi


[M::mp_idx_restore@11.240*0.60] loaded the index
[M::mp_idx_print_stat] 2347132 distinct k-mers; mean occ of infrequent k-mers: 329.05; 587 frequent k-mers accounting for 22261028 occurrences
[M::worker_pipeline::19.345*27.52] mapped 4066 sequences
[M::worker_pipeline::21.064*30.57] mapped 2919 sequences
[M::main] Version: 0.18-r281
[M::main] CMD: miniprot -S --gff-delim . --outc 0.8 -t 70 -I --gff-only /home/gyk/project/spider_silkome/data/raw/spider_genome/017.Hippasa_lycosina/Hippasa_lycosina.mpi /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_shortest_seq.fa
[M::main] Real time: 21.467 sec; CPU: 644.427 sec; Peak RSS: 7.830 GB
Processing genomes:  90%|█████████ | 9/10 [05:45<00:40, 40.93s/it]

2026-01-27 12:41:35.351 | INFO     | __main__:main:99 - Output: /home/gyk/project/spider_silkome/data/processed/miniprot_mapping_20260127/miniprot_output/Hippasa_lycosina/Hippasa_lycosina.gff
2026-01-27 12:41:35.352 | INFO     | __main__:main:89 - Processing Trichonephila_clavata: /home/gyk/project/spider_silkome/data/raw/spider_genome/119.Trichonephila_clavata/Trichonephila_clavata.mpi


[M::mp_idx_restore@3.893*1.00] loaded the index
[M::mp_idx_print_stat] 2364518 distinct k-mers; mean occ of infrequent k-mers: 391.92; 573 frequent k-mers accounting for 15240950 occurrences
[M::worker_pipeline::15.088*48.93] mapped 4066 sequences
[M::worker_pipeline::16.882*50.57] mapped 2919 sequences
[M::main] Version: 0.18-r281
[M::main] CMD: miniprot -S --gff-delim . --outc 0.8 -t 70 -I --gff-only /home/gyk/project/spider_silkome/data/raw/spider_genome/119.Trichonephila_clavata/Trichonephila_clavata.mpi /home/gyk/project/spider_silkome/data/interim/miniprot_mapping_20260127/cdhit/cdhit_shortest_seq.fa
[M::main] Real time: 17.346 sec; CPU: 854.098 sec; Peak RSS: 9.686 GB
Processing genomes: 100%|██████████| 10/10 [06:02<00:00, 36.28s/it]


2026-01-27 12:41:52.857 | INFO     | __main__:main:99 - Output: /home/gyk/project/spider_silkome/data/processed/miniprot_mapping_20260127/miniprot_output/Trichonephila_clavata/Trichonephila_clavata.gff
2026-01-27 12:41:52.858 | SUCCESS  | __main__:main:101 - Completed miniprot alignment for 10 genome(s)


In [ ]:
from pybedtools import BedTool

spider = "Trichonephila_clavata"
output_dir = f"{miniprot_output_path}/{spider}_all"
mRNA_gff = f"{output_dir}/{spider}.mRNA.gff"

merged_mRNA_gff = BedTool(mRNA_gff).merge(s=True)
for gene in merged_mRNA_gff:
    print(gene)


Defining the gene boundaries for each type of Spidroin should be part of the `for spider_genome in spider_genomes:` loop. For convenience, this section has been separated for testing.

<zh>处理每一种蛛丝蛋白的基因边界界定，这一部分应该作为`for spider_genome in spider_genomes:`循环下的一部分，这里为了方便，独立出来进行测试。</zh>

In [ ]:
spidroins = ['MaSp3', 'Pflag', 'MiSp', 'MaSp2', 'AcSp', 'CySp',
'MaSp3B', 'Spidroin', 'Putative_spidroin', 'CrSp', 'AgSp2', 'MaSp1',
'Ampullate_spidroin', 'MaSp', 'Flag', 'AgSp1', 'MaSp2B', 'PySp'] # Define the spidroins for test, should be the same as the spidroins used in the `for spider_genome in spider_genomes:` loop.
all_gff_records = []
all_csv = pd.DataFrame()
for spidroin in spidroins:
    # spidroin = "MiSp"
    spider = "Trichonephila_clavata"
    output_dir = f"{miniprot_output_path}/{spider}_all"
    spidroin_gff = f"{output_dir}/{spider}.mRNA.{spidroin}.gff"
    attr_dict = {}
    spidroin_gff_data = []
    with open(spidroin_gff, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue

            fields = line.strip().split('\t')

            # Analysis of the attributes field
            for attr in fields[8].split(';'):
                if '=' in attr:
                    key, value = attr.split('=', 1)
                    attr_dict[key] = value

            # Create attributes object
            attr_obj = Attributes(
                ID=attr_dict['ID'],
                Rank=int(attr_dict['Rank']),
                Identity=float(attr_dict['Identity']),
                Positive=float(attr_dict['Positive']),
                Target=attr_dict['Target'].split('|')
            )

            # Create gff_data object and add to list
            spidroin_gff_data.append(GFFData(
                seqid=fields[0],
                source=fields[1],
                type=fields[2],
                start=int(fields[3]),
                end=int(fields[4]),
                score=float(fields[5]),
                strand=fields[6],
                frame=fields[7],
                attributes=attr_obj
            ))

    # Sort by positive with descending order
    spidroin_gff_data.sort(key=lambda x: x.attributes.Positive, reverse=True)
    # Extract positions
    positions = extract_positions_from_gff(spidroin_gff_data, positive_threshold=0.75)
    # Export data
    # Export CSV data
    csv_output = spidroin_gff.replace('.gff', '.csv')
    df = positions_export(positions, csv_output, format='csv')
    all_csv = pd.concat([all_csv, df], ignore_index=True)

    # Export GFF data
    gff_output = spidroin_gff.replace('.gff', '.combined.gff')
    gff_records = positions_export(
        positions,
        gff_output,
        format='gff',
        spidroin=spidroin,
        min_length=1000, # Set minimum gene length threshold
        max_length=100000, # Set maximum gene length threshold
        extension_length=10000 # Set length to extend when start or end is missing
)

    all_gff_records.extend(gff_records)

# Convert to DataFrame and save
final_output_dir = f"{PROCESSED_DATA_DIR}/01.miniprot_mapping"
os.makedirs(final_output_dir, exist_ok=True)
if all_gff_records:
    df_combined = pd.DataFrame(all_gff_records)
    # Sort by seqid (chromosome) and start position with natural sorting
    df_combined['seqid_sort'] = df_combined['seqid'].str.extract(r'(\d+)').astype(float)
    df_combined = df_combined.sort_values(['seqid_sort', 'start']).drop('seqid_sort', axis=1)
    # //todo: 添加一个函数，能够处理 df_comined 中基因座相似的情况。
    # Write to GFF file
    with open(f"{final_output_dir}/{spider}.gff", 'w') as f:
        f.write("##gff-version 3\n")
        df_combined.to_csv(f, sep='\t', header=False, index=False)
else:
    print(f"No GFF records to combine for {spider}")

if not all_csv.empty:
    all_csv.to_csv(f"{final_output_dir}/{spider}.csv", index=False)

Filter positive_threshold > 0.75 mRNA in gff file

In [ ]:
spider_genomes = [f for f in os.listdir(spider_genome_path) if f.endswith(".fa.gz")]
positive_threshold = 0.75
for spider_genome in spider_genomes:
    spider = spider_genome.replace(".fa.gz", "")
    gff_file = f"{INTERIM_DATA_DIR}/miniprot/{spider}_all/{spider}.gff"
    if not os.path.exists(gff_file):
        gff_file = f"{INTERIM_DATA_DIR}/miniprot/{spider}_all/{spider}.fa.mRNA.gff"
    filtered_gff_file = f"{INTERIM_DATA_DIR}/miniprot/filtered_gff/{spider}.gff"
    os.makedirs(os.path.dirname(filtered_gff_file), exist_ok=True)
    cmd = f"awk -F'\t' '$3==\"mRNA\"{{match($9,/Positive=([0-9.]+)/,m); if(m[1]>={positive_threshold}) print; next}}1' {gff_file} | sort -k1,1V -k4,4n -k9,9 > {filtered_gff_file}"
    run_shell_command_with_check(cmd, filtered_gff_file, force=True)
